#### Loading Star Catalogs
Catalogs can be downloaded from a variety of services like VizieR, typically consist of a data file and a ReadMe, explaining its formatting. Below, I will discuss a couple of ways to load these data files and more optimal ways to store them.

We'll start with basic imports, and the `astropy` provided method of loading. 

In [1]:
from astropy.io import ascii
from astropy.table import Table, vstack
import numpy as np
import time
import numpy as np
import pandas as pd

In [ ]:
# Uses Astropy to read the Tycho catalog
### LARGE catalog, and its currently unoptimized, so it will take awhile to load. 
#tycho = ascii.read('./catalog/tyc_main.dat', readme="./catalog/ReadMe")
#print(f"Loaded {len(tycho)} Tycho objects")
#print(tycho.colnames)

Astropy also provides a method for loading catalog into a table, which is slightly faster and more efficient.

In [33]:
tycho = Table.read('../data/tyc_main.dat', format='ascii', readme="../data/ReadMe")

In [7]:
# get columns
print(tycho.colnames)
print(tycho[0])

['Catalog', 'TYC', 'Proxy', 'RAhms', 'DEdms', 'Vmag', '---', 'r_Vmag', 'RAdeg', 'DEdeg', 'AstroRef', 'Plx', 'pmRA', 'pmDE', 'e_RAdeg', 'e_DEdeg', 'e_Plx', 'e_pmRA', 'e_pmDE', 'DE:RA', 'Plx:RA', 'Plx:DE', 'pmRA:RA', 'pmRA:DE', 'pmRA:Plx', 'pmDE:RA', 'pmDE:DE', 'pmDE:Plx', 'pmDE:pmRA', 'Nastro', 'F2', 'HIP', 'BTmag', 'e_BTmag', 'VTmag', 'e_VTmag', 'r_BTmag', 'B-V', 'e_B-V', '---_1', 'Q', 'Fs', 'Source', 'Nphoto', 'VTscat', 'VTmax', 'VTmin', 'Var', 'VarFlag', 'MultFlag', 'morePhoto', 'm_HIP', 'PPM', 'HD', 'BD', 'CoD', 'CPD', 'Remark']
Catalog    TYC    Proxy    RAhms       DEdms    Vmag --- r_Vmag   RAdeg      DEdeg    AstroRef Plx   pmRA     pmDE   e_RAdeg e_DEdeg e_Plx  e_pmRA   e_pmDE  DE:RA Plx:RA Plx:DE pmRA:RA pmRA:DE pmRA:Plx pmDE:RA pmDE:DE pmDE:Plx pmDE:pmRA Nastro   F2  HIP BTmag  e_BTmag VTmag e_VTmag r_BTmag B-V  e_B-V ---_1  Q   Fs  Source Nphoto VTscat VTmax VTmin Var VarFlag MultFlag morePhoto m_HIP  PPM     HD       BD    CoD CPD Remark
                                    

From a `Table`, we can save it directly to a `.fits` file format, which is **MUCH** faster to load. 
Beforehand, we can remove unncessary columns to also increase speed. 

In [34]:
col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'HD', 'HIP']
tycho = tycho[col_names]
tycho.write('../data/tyc_build.fits', format='fits', overwrite=True)

For sky navigation, we'd like some identification for useful, bright stars. 
We will add an extra `Name` column to our fits data file.
We will use constellation-based names like "33 Psc" as well as coloquial names like Rigel or Polaris.

In [35]:
tycho = Table.read('../data/tyc_build.fits')
common_names = Table.read("../data/index.dat", format='ascii', readme="../data/ReadMe_names")
bright_stars = Table.read('../data/catalog', format='ascii', readme="../data/ReadMe_bright")

Loop through every entry in Tycho, and look for a name in the other two catalogs by HD number id.

In [ ]:
# create name column in tycho
tycho['Name'] = np.full(len(tycho), '', dtype='U50')
# loop through tycho catalog. look for HD number in common names, then bright stars
for i, row in enumerate(tycho):
    hd_number = row['HD']
    if hd_number == 0:
        continue

    name = ""
    # Check if HD number exists in common names or bright stars
    if hd_number in common_names['HD']:
        common = common_names[common_names['HD'] == hd_number]['name'][0]
        common = str(common).replace('--', '').strip()
        if len(common) > 0:
            name = common
    elif hd_number in bright_stars['HD']:
        bright = bright_stars[bright_stars['HD'] == hd_number]['Name'][0]
        bright = str(bright).replace('--', '').strip()
        if len(bright) > 0:
            name = bright
    tycho['Name'][i] = name
    if len(name) > 0:
        print(f"Found name for HD {hd_number}: {name}")

# Save the updated tycho table with names
tycho.write('../data/tyc_build.fits', format='fits', overwrite=True)

Now that our catalog has names, lets remove unnecessary columns so loading and searching is faster.

In [11]:
tycho = Table.read('./catalog/tyc.fits')
col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'Name']
tycho = tycho[col_names]
tycho.write('./catalog/tyc.fits', format='fits', overwrite=True)

Lets see whats in our catalog now.

In [37]:
tycho = Table.read('../data/tyc_build.fits')
# get brighest 10 stars
brightest = tycho[np.argsort(tycho['Vmag'])[:100]]
print("Brightest 10 stars:")
for star in brightest:
    print(f"TYC: {star['TYC']}, Vmag: {star['Vmag']}, Name: {star['Name']}")

Brightest 10 stars:
TYC: 5949  2777 1, Vmag: -1.44, Name: SIRIUS
TYC: 8534  2277 1, Vmag: -0.63, Name: CANOPUS
TYC: 9007  5849 1, Vmag: -0.01, Name: RIGEL KENTAURUS
TYC: 3105  2070 1, Vmag: 0.03, Name: VEGA
TYC: 3358  3141 1, Vmag: 0.08, Name: CAPELLA
TYC: 1472  1436 1, Vmag: 0.16, Name: ARCTURUS
TYC: 5331  1752 1, Vmag: 0.28, Name: RIGEL
TYC: 187  2184 1, Vmag: 0.4, Name: PROCYON
TYC: 8478  1395 1, Vmag: 0.54, Name: ACHERNAR
TYC: 129  1873 1, Vmag: 0.57, Name: BETELGEUSE
TYC: 9005  3919 1, Vmag: 0.64, Name: --
TYC: 1058  3399 1, Vmag: 0.93, Name: ALTAIR
TYC: 1266  1416 1, Vmag: 0.99, Name: ALDEBARAN
TYC: 5547  1518 1, Vmag: 1.06, Name: SPICA
TYC: 6803  2158 1, Vmag: 1.07, Name: ANTARES
TYC: 1920  2194 1, Vmag: 1.22, Name: POLLUX
TYC: 6977  1267 1, Vmag: 1.23, Name: FOMALHAUT
TYC: 8979  3464 1, Vmag: 1.28, Name: ACRUX
TYC: 8659  3107 1, Vmag: 1.31, Name: --
TYC: 3574  3347 1, Vmag: 1.33, Name: DENEB
TYC: 9007  5848 1, Vmag: 1.35, Name: Alp2Cen
TYC: 833  1381 1, Vmag: 1.41, Name: REGULU

We are still missing names for some of our brightest stars, lets use another catalog, Hipparcos. 
For every Tycho entry above a threshold brightness and without a HD number, we will look for its TYC ID in Hipparcos to find a correct HD number. We can then search for a name in common names and bright stars. 

In [36]:
tycho = Table.read('../data/tyc_build.fits')
hipparcos = Table.read("../data/hip_main.dat", format='ascii', readme="../data/ReadMe")

# Filter Vmag < 6.0 and no HD number
#tycho_short = tycho[(tycho['Vmag'] < 6.0) & (tycho['Name'].mask)]

# create name column in tycho short
tycho['Name'] = np.full(len(tycho), '', dtype='U50')
# loop through tycho catalog. look for HD number in common names, then bright stars
for i, row in enumerate(tycho):
    name = ""
    hd_number = row['HD']
    mag = row['Vmag']
    if mag > 8.0:
        continue
    if not isinstance(hd_number, np.int64):
        hip_number = row['HIP']

        hip = hipparcos[hipparcos['HIP'] == hip_number]
        if len(hip) == 0:
            continue
        else:
            hd_number = hip['HD'][0]
    
    if hd_number in common_names['HD']:
            common = common_names[common_names['HD'] == hd_number]['name'][0]
            common = str(common).replace('--', '').strip()
            if len(common) > 0:
                name = common
    elif hd_number in bright_stars['HD']:
        bright = bright_stars[bright_stars['HD'] == hd_number]['Name'][0]
        bright = str(bright).replace('--', '').strip()
        if len(bright) > 0:
            name = bright

    
    tycho['Name'][i] = name
    if len(name) > 0:
        print(f"Found{name}")

col_names = ['RAdeg', 'DEdeg', 'TYC', 'Vmag', 'Name']
tycho = tycho[col_names]
tycho.write('../data/tyc_build.fits', format='fits', overwrite=True)

Found51    Psc
Found60    Psc
Found96 G  PSC
Found62    Psc
Found33    Cet
Found77    Psc
Found77    Psc
Found73    Psc
Found80    Psc
Found88    Psc
Found98Mu  Psc
Found60    Cet
Found69    Cet
Found112    Psc
Found113Alp Psc
Found113Alp Psc
Found86Gam Cet
Found86Gam Cet
Found268 G CET
Found93    Cet
FoundMENKAR
Found97Kap2Cet
Found12    Tau
Found31    Tau
Found40    Tau
Found45    Tau
Found10Pi 6Ori
Found5    Ori
Found25Psi1Ori
Found17Rho Ori
Found21    Ori
Found23    Ori
Found30Psi2Ori
Found30Psi2Ori
FoundBELLATRIX
Found51    Ori
Found56    Ori
Found59    Ori
Found33    Ori
Found33    Ori
Found38    Ori
Found47Ome Ori
Found32    Ori
Found52    Ori
FoundBETELGEUSE
Found63    Ori
Found12    Mon
Found5Eta CMi
Found9Del3CMi
Found7Del1CMi
Found8Del2CMi
Found14    CMi
FoundPROCYON
Found7Eta Hya
Found10    Hya
Found11Eps Hya
Found11Eps Hya
Found13Rho Hya
Found18Ome Hya
Found7    Sex
Found4    Sex
Found10    Leo
FoundRS SEX
Found13    Sex
Found19    Sex
Found14    Sex
Found43    Leo
Found36

We can now add methods for searching directly for stars by name or coordinates.

In [38]:
tycho = Table.read('../data/tyc.fits')

def clean_name(n) -> str:
    if n is np.ma.masked:
        return ''
    n = str(n)
    n = n.strip()
    return n.lower()

def search_by_name(n: str):
    n = clean_name(n)
    name_col = tycho['Name'].filled('')
    name_strings = np.array([clean_name(n) for n in name_col])

    results = tycho[name_strings == n]
    print(len(results))

    return results

def search_by_coordinate(ra: float, dec: float, radius: float = 0.1):
    ra = float(ra)
    dec = float(dec)
    radius = float(radius)

    # Convert radius from degrees to radians
    radius_rad = np.radians(radius)

    # Calculate the distance in degrees
    delta_ra = np.radians(tycho['RAdeg'] - ra)
    delta_dec = np.radians(tycho['DEdeg'] - dec)

    # Haversine formula to calculate distance
    a = (np.sin(delta_dec / 2) ** 2 +
         np.cos(np.radians(dec)) * np.cos(np.radians(tycho['DEdeg'])) *
         np.sin(delta_ra / 2) ** 2)
    c = 2 * np.arcsin(np.sqrt(a))

    # Distance in radians
    distance_rad = c

    # Filter results within the specified radius
    results = tycho[distance_rad <= radius_rad]
    
    return results

target = search_by_name("pleiades")
print(f"{target}")

near = search_by_coordinate(target['RAdeg'], target['DEdeg'])
print(near)



1
RAdeg DEdeg  TYC Vmag   Name  
 deg   deg       mag          
----- ------ --- ---- --------
3.783 24.117  OC  1.2 Pleiades
  RAdeg       DEdeg        TYC      Vmag   Name  
   deg         deg                  mag          
---------- ----------- ------------ ---- --------
3.86066522 24.06974512 1730  1332 1 9.47       --
     3.783      24.117           OC  1.2 Pleiades


C:\Users\314ow\AppData\Local\Temp\ipykernel_28424\2143646842.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ra = float(ra)
C:\Users\314ow\AppData\Local\Temp\ipykernel_28424\2143646842.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dec = float(dec)


In [20]:
def ra_to_deg(ra):
    if isinstance(ra, str):
        h, m, s = [float(x) for x in ra.split(':')]
        return 15 * (h + m / 60 + s / 3600)  # 1h = 15°
    return np.nan

def dec_to_deg(dec):
    if isinstance(dec, str):
        sign = -1 if dec.strip().startswith('-') else 1
        d, m, s = [float(x) for x in dec.strip().replace('-', '').split(':')]
        return sign * (d + m / 60 + s / 3600)
    return np.nan

In [39]:
messier = pd.read_csv('../data/catalogue-de-messier.csv', sep=";")
messier['RAdeg'] = messier['RA (Right Ascension)'].apply(ra_to_deg)
messier['DEdeg'] = messier['Dec (Declinaison)'].apply(dec_to_deg)
messier['TYC'] = messier['Messier']
messier['Vmag'] = messier['Magnitude']
messier['Name'] = ""

col_names = ['RAdeg', 'DEdeg', 'Constellation', 'Vmag', 'Name', 'TYC']
messier = messier[col_names]

# manually rename named messier objects
messier.loc[messier['TYC'] == 'M1', 'Name'] = 'Crab Nebula'
messier.loc[messier['TYC'] == 'M4', 'Name'] = 'Spider Globular CLuster'
messier.loc[messier['TYC'] == 'M5', 'Name'] = 'Rose Cluster'
messier.loc[messier['TYC'] == 'M6', 'Name'] = 'Butterfly Cluster'
messier.loc[messier['TYC'] == 'M7', 'Name'] = 'Ptolemy\'s Cluster'
messier.loc[messier['TYC'] == 'M8', 'Name'] = 'Lagoon Nebula'
messier.loc[messier['TYC'] == 'M11', 'Name'] = 'Wild duck Cluster'
messier.loc[messier['TYC'] == 'M13', 'Name'] = 'Hercules Cluster'
messier.loc[messier['TYC'] == 'M16', 'Name'] = 'Peagasus Cluster'
messier.loc[messier['TYC'] == 'M16', 'Name'] = 'Eagle Nebula'
messier.loc[messier['TYC'] == 'M17', 'Name'] = 'Horseshoe'
messier.loc[messier['TYC'] == 'M18', 'Name'] = 'Black Swan Cluster'
messier.loc[messier['TYC'] == 'M20', 'Name'] = 'Trifid Nebula'
messier.loc[messier['TYC'] == 'M21', 'Name'] = 'Webb\'s Cross Cluster'
messier.loc[messier['TYC'] == 'M22', 'Name'] = 'Sagittarius Cluster'
messier.loc[messier['TYC'] == 'M24', 'Name'] = 'Sagittarius Cloud'
messier.loc[messier['TYC'] == 'M27', 'Name'] = 'Dumbbell Nebula'
messier.loc[messier['TYC'] == 'M29', 'Name'] = 'Cooling Tower Cluster'
messier.loc[messier['TYC'] == 'M30', 'Name'] = 'Jellyfish Cluster'
messier.loc[messier['TYC'] == 'M31', 'Name'] = 'Andromeda Galaxy'
messier.loc[messier['TYC'] == 'M33', 'Name'] = 'Triangulum'
messier.loc[messier['TYC'] == 'M42', 'Name'] = 'Orion Nebula'
messier.loc[messier['TYC'] == 'M44', 'Name'] = 'Beehive Cluster'
messier.loc[messier['TYC'] == 'M45', 'Name'] = 'Pleiades'
messier.loc[messier['TYC'] == 'M51', 'Name'] = 'Whirlpool Galaxy'
messier.loc[messier['TYC'] == 'M57', 'Name'] = 'Ring Nebula'
messier.loc[messier['TYC'] == 'M81', 'Name'] = 'Bode\'s Galaxy'
messier.loc[messier['TYC'] == 'M82', 'Name'] = 'Cigar Galaxy'
messier.loc[messier['TYC'] == 'M87', 'Name'] = 'Virgo A'
messier.loc[messier['TYC'] == 'M101', 'Name'] = 'Pinwheel Galaxy'
messier.loc[messier['TYC'] == 'M104', 'Name'] = 'Sombrero Galaxy'

# Add data to tyc_build.fits
tycho = Table.read('../data/tyc_build.fits')
messier = Table.from_pandas(messier)
messier['Name'] = messier['Name'].astype('U50')
tycho = vstack([tycho, messier])
tycho.write('../data/tyc_build.fits', format='fits', overwrite=True)

In [42]:
tycho = Table.read('../data/tyc_build.fits')
mask = [str(t['TYC'])[0] == 'M' for t in tycho]
print(tycho[mask])
print(len(tycho[mask]))

      RAdeg                DEdeg         ... Constellation
       deg                  deg          ...              
------------------ --------------------- ... -------------
179.39991666666666     53.37452777777778 ...           UMa
10.091999999999999     41.68530555555555 ...           And
 83.63320833333333     22.01447222222222 ...           Tau
325.09175000000005   -23.179083333333335 ...           Cap
251.81050000000002   -1.9478333333333333 ...           Oph
 83.81866666666666    -5.389666666666667 ...           Ori
351.20166666666665     61.59316666666667 ...           Cas
114.14591666666666   -14.482611111111112 ...           Pup
264.40066666666667   -3.2459166666666666 ...           Oph
               ...                   ... ...           ...
245.89749999999998    -26.52552777777778 ...           Sco
 92.27108333333334    24.338638888888887 ...           Gem
         40.669625 -0.013277777777777777 ...           Cet
198.95554166666668     42.02927777777778 ...           C